## Function to downsample the dataset, run a GridSearch, sort the best model according to the mean average percentage error

* Downsampling of dataset: Pick 10 days in a device-specific dataset and will run the GridSearch. Allow to run the algorithm on all device-specific dataframes.
* GridSearch trying Prophet with different training periods (8, 10 or 12 training days). This was the most critical parameter affecting the mean average percentage error (mape).
* Sort the Prophet model according to the mape. Save the best model with graph and a dataframe containing the prediction and actual data.

## 1 - Import the relevant library

In [1]:
import pandas as pd
import numpy as np
import time
import re

import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

# Note that the interactive plot may not work in Jupyter lab, but only in Jupyter Notebook (conflict of javascripts)
%matplotlib widget 

from datetime import datetime, timedelta
from pytz import timezone

In [2]:
import fbprophet
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric

In [3]:
fbprophet.__version__

'0.6'

In [4]:
from sklearn.model_selection import ParameterGrid
import itertools
from random import sample

In [5]:
# Import the functions from the helper.py
from helper import load_ds, df_dev_formater, find_index, df_generator, prophet_fit, prophet_plot, get_outliers, prophet, GridSearch_Prophet

In [6]:
n_samples = 10 # Limit to 10 predictions per device.
pred_duration = 12 # 12-day prediction

for dev_nb in range(1,52):
    device_nb = str('{:02d}'.format(dev_nb))
    # Load the device-specific dataframe.
    assert isinstance(device_nb, str) and len(device_nb)==2 and sum(d.isdigit() for d in device_nb)==2, 'WARNING: device_nb must be a string of 2-digits!'
    assert int(device_nb)>=1 and int(device_nb)<=51, 'This device does not belong to the dataframe'
    device, df_dev = load_ds(device_nb)
    # Convert the variable device from a np.array to a string
    regex = re.compile('[^A-Za-z0-9]')
    device = regex.sub('', str(device))
    
    # Create a dataframe with the dates to use
    dates = pd.DataFrame(columns={'date_minus_12',
                                  'date_minus_10',
                                  'date_minus_8',
                                  'date_predict'})
    dates = dates[['date_minus_12', 'date_minus_10', 'date_minus_8', 'date_predict']]
    # List of unique dates in the dataframe
    dates['date_minus_12'] = df_dev['ds'].unique().strftime('%Y-%m-%d')
    dates = dates.drop_duplicates(subset=['date_minus_12'])
    dates = dates.reset_index(drop=True)
    # Fill the other columns and drop the 12 last columns
    dates['date_minus_10'] = dates.iloc[2:, 0].reset_index(drop=True)
    dates['date_minus_8'] = dates.iloc[4:, 0].reset_index(drop=True)
    dates['date_predict'] = dates.iloc[12:, 0].reset_index(drop=True)
    dates = dates[:-pred_duration] # Drop the 12 last rows
    
    # Keep only the dates with at least 12 training days
    dates['Do_It'] = 'Do not'
    dates['dm_12_c'] = np.nan
    
    for r in range(dates.shape[0]):
        # Calculate the date_predict - pred_duration
        date_predict = dates.iloc[r, 3]
        date_predict = datetime.strptime(date_predict, "%Y-%m-%d")

        date_minus_12_check = date_predict + timedelta(days=-pred_duration)
        date_minus_12_check = datetime.strftime(date_minus_12_check, "%Y-%m-%d")
        
        # Tag the date_predict that have at least 12 training days
        if date_minus_12_check in dates.date_predict.values or r<=11:
            dates.iloc[r, 4] = 'yes'

    dates = dates[dates.Do_It == 'yes']
    dates.drop(['Do_It', 'dm_12_c'], axis=1)
    
    # Downsampling
    if dates.shape[0]>n_samples:
        dates = dates.sample(n=n_samples, replace=False)

    # GridSearch over the (down-sampled) dataset:
    start_time = time.time()
    mape_table_full = pd.DataFrame()
    
    for r in range(dates.shape[0]):
        # Parameters of the Grid
        prophet_grid = {'df_dev' : [df_dev],
                        'device' : [device],
                        'parameter' : ['co2'],
                        'begin' : dates.iloc[r, :3].tolist(),
                        'end' : [dates.iloc[r, 3]],
                        'sampling_period_min' : [1],
                        'graph' : [1],
                        'predict_day' : [1],
                        'interval_width' : [0.6],
                        'changepoint_prior_scale' : [0.01, 0.005], # list(np.arange(0.01,30,1).tolist()),
                        'daily_fo' : [3],
            #             'holidays_prior_scale' : list((1000,100,10,1,0.1)),
                           }

        # Run GridSearch_Prophet
        mape_table = GridSearch_Prophet(list(ParameterGrid(prophet_grid)), metric='mape')
        mape_table_full = mape_table_full.append(mape_table)

        end_time = time.time()
        dur_min = int((end_time - start_time)/60)
        
        print('Time elapsed: '+ str(dur_min) + " minutes.")

        # Save the best model
        print('Saving the best model')
        
        best_model = {'df_dev' : [df_dev],
                      'device' : [mape_table.iloc[0, 0]],
                      'parameter' : [mape_table.iloc[0, 1]],
                      'begin' : [mape_table.iloc[0, 2]],
                      'end' : [mape_table.iloc[0, 3]],
                      'sampling_period_min' : [mape_table.iloc[0, 4]],
                      'graph' : [1],
                      'predict_day' : [1],
                      'interval_width' : [mape_table.iloc[0, 5]],
                      'changepoint_prior_scale' : [mape_table.iloc[0, 7]], # list(np.arange(0.01,30,1).tolist()),
                      'daily_fo' : [mape_table.iloc[0, 6]],
#                       'holidays_prior_scale' : list((1000,100,10,1,0.1)),
                           }

        # Run GridSearch_Prophet on the best model
        mape_table = GridSearch_Prophet(list(ParameterGrid(best_model)), metric='mape')

        end_time = time.time()
        dur_min = int((end_time - start_time)/60)
        print('Full analysis completed in '+ str(dur_min) + ' minutes.')

    # Save the full table of mape_table
    # Store the complete mape_table if this is the last prediction
    folder_name = '/Users/path_to_your_folder/data/processed/'
    mape_table_name = folder_name + re.sub("[']", '', str(mape_table.iloc[0, 0])) + '_mape_table_full.csv'
    mape_table_full.to_csv(mape_table_name)

Check report:
##############################################
['Device contained in the dataset: device45']
['Tenant using the device: tenant08']

Data types:
device                                object
tenant                                object
ds             datetime64[ns, Europe/Zurich]
light                                float64
temperature                          float64
humidity                             float64
co2                                  float64
dtype: object

Available data from the 2019-10-23 to the 2020-03-26.

prophet_instance nb 0
Full dataset: 2019-10-23 to the 2020-03-26. Analysed data the 2020-03-06 to the 2020-03-18.
o Trained on the data from the 2020-03-06 to the 2020-03-17 (10 days).
o Predict from the 2020-03-17 to the 2020-03-18 (1 days).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


prophet_instance nb 1
Full dataset: 2019-10-23 to the 2020-03-26. Analysed data the 2020-03-06 to the 2020-03-18.
o Trained on the data from the 2020-03-06 to the 2020-03-17 (10 days).
o Predict from the 2020-03-17 to the 2020-03-18 (1 days).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …


prophet_instance nb 2
Full dataset: 2019-10-23 to the 2020-03-26. Analysed data the 2020-03-08 to the 2020-03-18.
o Trained on the data from the 2020-03-08 to the 2020-03-17 (8 days).
o Predict from the 2020-03-17 to the 2020-03-18 (1 days).


...


prophet_instance nb 5
Full dataset: 2019-10-30 to the 2020-03-26. Analysed data the 2019-12-20 to the 2019-12-28.
o Trained on the data from the 2019-12-20 to the 2019-12-27 (6 days).
o Predict from the 2019-12-27 to the 2019-12-28 (1 days).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Time elapsed: 33 minutes.
Saving the best model

prophet_instance nb 0
Full dataset: 2019-10-30 to the 2020-03-26. Analysed data the 2019-12-16 to the 2019-12-28.
o Trained on the data from the 2019-12-16 to the 2019-12-27 (10 days).
o Predict from the 2019-12-27 to the 2019-12-28 (1 days).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Full analysis completed in 33 minutes.
